In [16]:
# import cv2
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
from pca import pca
from sklearn.decomposition import PCA

In [4]:
# Cargar el dataset de imagenes reducidas con PCA
df_reduced = pd.read_csv(r'3_Reduced_dataset\X_reduced.csv')

In [ ]:
plt.imshow(np.array(df_reduced[0:1]).reshape(398, 188), cmap='gray')
plt.show()

In [18]:
imagen_reconstruida = PCA.inverse_transform(np.array(df_reduced[0:1]))

TypeError: _BasePCA.inverse_transform() missing 1 required positional argument: 'X'

In [7]:
# Tomamos la primer imagen
image = df_reduced[0:1]

# Definir características (intensidad de píxeles)
caracteristicas = image.reshape(-1, 1)

# Crear y ajustar el modelo DBSCAN
dbscan = DBSCAN(eps=5, min_samples=500)  # Ajusta los parámetros según tus necesidades
etiquetas = dbscan.fit_predict(caracteristicas)

# Identificar grupos y puntos ruidosos
areas_densas = caracteristicas[etiquetas != -1]
puntos_ruidosos = caracteristicas[etiquetas == -1]

# Visualizar las áreas densas en la imagen original
imagen_resultado = image.copy()
imagen_resultado[etiquetas == -1] = 0  # Establece puntos ruidosos como negro en la imagen resultado

# Mostrar la imagen original y la imagen con las áreas densas resaltadas
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title("Imagen Original")
plt.imshow(image, cmap='gray')

AttributeError: 'DataFrame' object has no attribute 'reshape'

In [ ]:
# Cargar la imagen de radiografía dental (asegúrate de tener OpenCV instalado)
imagen = cv2.imread('radiografia_dental.jpg', 0)  # Carga la imagen en escala de grises

# Definir características (intensidad de píxeles)
caracteristicas = imagen.reshape(-1, 1)

# Crear y ajustar el modelo DBSCAN
dbscan = DBSCAN(eps=5, min_samples=500)  # Ajusta los parámetros según tus necesidades
etiquetas = dbscan.fit_predict(caracteristicas)

# Identificar grupos y puntos ruidosos
areas_densas = caracteristicas[etiquetas != -1]
puntos_ruidosos = caracteristicas[etiquetas == -1]

# Visualizar las áreas densas en la imagen original
imagen_resultado = imagen.copy()
imagen_resultado[etiquetas == -1] = 0  # Establece puntos ruidosos como negro en la imagen resultado

# Mostrar la imagen original y la imagen con las áreas densas resaltadas
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title("Imagen Original")
plt.imshow(imagen, cmap='gray')

plt.subplot(1, 2, 2)
plt.title("Áreas Densas Resaltadas")
plt.imshow(imagen_resultado, cmap='gray')

plt.tight_layout()
plt.show()